<a href="https://colab.research.google.com/github/tsaiton123/RL/blob/main/meta_on_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [3]:
!apt-get update && sudo apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,016 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,284 kB]
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubun

# Environment Setup

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])

import itertools
# Add imports for meta-learning and reinforcement learning libraries
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import glob, random
from collections import OrderedDict

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"DEVICE = {device}")
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


DEVICE = cpu


# Data Acquisition and Preprocessing

In [5]:
config_tickers.DOW_30_TICKER

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [6]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [7]:
df_raw = YahooDownloader(start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, ticker_list=config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

# Feature Engineering

In [8]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False
)
df_processed = fe.preprocess_data(df_raw)
df_processed.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (3228, 8)
Successfully added vix
Successfully added turbulence index


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.747389,746015200,AAPL,4,0.0,2.969346,2.641384,100.0,66.666667,100.0,2.747389,2.747389,39.189999,0.0
1,2009-01-02,58.590000,59.080002,57.750000,42.737892,6547900,AMGN,4,0.0,2.969346,2.641384,100.0,66.666667,100.0,42.737892,42.737892,39.189999,0.0
2,2009-01-02,18.570000,19.520000,18.400000,15.193853,10955700,AXP,4,0.0,2.969346,2.641384,100.0,66.666667,100.0,15.193853,15.193853,39.189999,0.0
3,2009-01-02,42.799999,45.560001,42.779999,33.941090,7010200,BA,4,0.0,2.969346,2.641384,100.0,66.666667,100.0,33.941090,33.941090,39.189999,0.0
4,2009-01-02,44.910000,46.980000,44.709999,31.093393,7117200,CAT,4,0.0,2.969346,2.641384,100.0,66.666667,100.0,31.093393,31.093393,39.189999,0.0


In [9]:
list_ticker = df_processed["tic"].unique().tolist()
list_date = list(pd.date_range(df_processed['date'].min(), df_processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))

processed_full = pd.DataFrame(combination, columns=["date","tic"]).merge(df_processed, on=["date","tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(df_processed['date'])]
processed_full = processed_full.sort_values(['date', 'tic'])
processed_full = processed_full.fillna(0)
processed_full

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.747389,746015200.0,4.0,0.000000,2.969346,2.641384,100.000000,66.666667,100.000000,2.747389,2.747389,39.189999,0.000000
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,42.737892,6547900.0,4.0,0.000000,2.969346,2.641384,100.000000,66.666667,100.000000,42.737892,42.737892,39.189999,0.000000
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.193853,10955700.0,4.0,0.000000,2.969346,2.641384,100.000000,66.666667,100.000000,15.193853,15.193853,39.189999,0.000000
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941090,7010200.0,4.0,0.000000,2.969346,2.641384,100.000000,66.666667,100.000000,33.941090,33.941090,39.189999,0.000000
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.093393,7117200.0,4.0,0.000000,2.969346,2.641384,100.000000,66.666667,100.000000,31.093393,31.093393,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135773,2021-10-27,UNH,454.640015,460.440002,453.480011,441.170837,3520400.0,2.0,11.345162,451.559682,362.173189,65.085975,174.351128,51.571330,403.881877,404.166418,16.980000,145.285741
135774,2021-10-27,V,224.750000,224.750000,215.660004,212.099335,22958100.0,2.0,0.013798,232.994211,214.822632,41.737207,-153.682033,30.337021,223.018857,225.227715,16.980000,145.285741
135775,2021-10-27,VZ,53.169998,53.200001,52.470001,46.484344,15007400.0,2.0,-0.215911,48.194482,45.191918,44.604904,-50.004109,8.504850,46.923545,47.527330,16.980000,145.285741
135776,2021-10-27,WBA,48.450001,48.459999,47.090000,41.611912,5652000.0,2.0,-0.014508,44.215244,40.487785,46.167771,-62.027236,5.045608,42.551244,42.866390,16.980000,145.285741


In [10]:
df_train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
df_test = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)
print(len(df_train))
print(len(df_test))

83897
9715


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Separate different stocks

In [11]:
# Extracting the unique tickers from the dataframe
unique_tickers = df_train['tic'].unique()

# Creating an array of dataframes, each containing data for a specific ticker
dataframes_array_train = [df_train[df_train['tic'] == ticker] for ticker in unique_tickers]

# Displaying the tickers and the corresponding index in the array
ticker_index_mapping = {ticker: index for index, ticker in enumerate(unique_tickers)}
ticker_index_mapping


{'AAPL': 0,
 'AMGN': 1,
 'AXP': 2,
 'BA': 3,
 'CAT': 4,
 'CRM': 5,
 'CSCO': 6,
 'CVX': 7,
 'DIS': 8,
 'GS': 9,
 'HD': 10,
 'HON': 11,
 'IBM': 12,
 'INTC': 13,
 'JNJ': 14,
 'JPM': 15,
 'KO': 16,
 'MCD': 17,
 'MMM': 18,
 'MRK': 19,
 'MSFT': 20,
 'NKE': 21,
 'PG': 22,
 'TRV': 23,
 'UNH': 24,
 'V': 25,
 'VZ': 26,
 'WBA': 27,
 'WMT': 28}

In [14]:
dataframes_array_train[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.747389,7.460152e+08,4.0,0.000000,2.969346,2.641384,100.000000,66.666667,100.000000,2.747389,2.747389,39.189999,0.000000
1,2009-01-05,AAPL,3.327500,3.435000,3.311071,2.863341,1.181608e+09,0.0,0.002601,2.969346,2.641384,100.000000,66.666667,100.000000,2.805365,2.805365,39.080002,0.000000
2,2009-01-06,AAPL,3.426786,3.470357,3.299643,2.816114,1.289310e+09,1.0,0.001850,2.925562,2.692333,70.355953,46.865825,100.000000,2.808948,2.808948,38.560001,0.000000
3,2009-01-07,AAPL,3.278929,3.303571,3.223571,2.755262,7.530488e+08,2.0,-0.000734,2.904834,2.686219,50.429535,-29.676748,43.607834,2.795526,2.795526,43.389999,0.000000
4,2009-01-08,AAPL,3.229643,3.326786,3.215714,2.806425,6.735008e+08,3.0,-0.000087,2.892870,2.702542,60.227194,-9.098859,48.357918,2.797706,2.797706,42.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,2020-06-24,AAPL,91.250000,92.197502,89.629997,88.141586,1.926232e+08,2.0,3.283722,90.666270,75.768213,63.765191,135.613604,38.191255,81.123168,74.731048,33.840000,10.120711
2889,2020-06-25,AAPL,90.175003,91.250000,89.392502,89.311684,1.375224e+08,3.0,3.330736,91.270863,76.304126,64.828578,123.656579,36.230796,81.589832,75.184887,32.220001,22.285927
2890,2020-06-26,AAPL,91.102501,91.330002,88.254997,86.567513,2.052568e+08,4.0,3.110705,91.249594,77.199072,60.519605,94.940087,27.005877,81.949600,75.647432,34.730000,68.833909
2891,2020-06-29,AAPL,88.312500,90.542503,87.820000,88.562622,1.306460e+08,0.0,3.062021,91.501937,77.924203,62.399277,92.929893,23.584901,82.390806,76.126872,31.780001,36.823425


In [13]:
# Extracting the unique tickers from the dataframe
unique_tickers = df_test['tic'].unique()

# Creating an array of dataframes, each containing data for a specific ticker
dataframes_array_test = [df_test[df_test['tic'] == ticker] for ticker in unique_tickers]

# Displaying the tickers and the corresponding index in the array
ticker_index_mapping = {ticker: index for index, ticker in enumerate(unique_tickers)}
ticker_index_mapping


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'AAPL': 0,
 'AMGN': 1,
 'AXP': 2,
 'BA': 3,
 'CAT': 4,
 'CRM': 5,
 'CSCO': 6,
 'CVX': 7,
 'DIS': 8,
 'GS': 9,
 'HD': 10,
 'HON': 11,
 'IBM': 12,
 'INTC': 13,
 'JNJ': 14,
 'JPM': 15,
 'KO': 16,
 'MCD': 17,
 'MMM': 18,
 'MRK': 19,
 'MSFT': 20,
 'NKE': 21,
 'PG': 22,
 'TRV': 23,
 'UNH': 24,
 'V': 25,
 'VZ': 26,
 'WBA': 27,
 'WMT': 28}

In [15]:
dataframes_array_test[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,89.132996,110737200.0,2.0,2.988729,91.903725,79.491803,62.807135,107.505445,29.730532,83.213404,77.050532,28.620001,53.068061
1,2020-07-02,AAPL,91.962502,92.617500,90.910004,89.132996,114041600.0,3.0,2.908266,91.702605,80.715929,62.807135,104.545219,33.335609,83.579620,77.480479,27.680000,6.839125
2,2020-07-06,AAPL,92.500000,93.945000,92.467499,91.517326,118655600.0,0.0,3.002286,92.167184,81.288065,65.035231,124.272838,39.043351,84.044735,77.923146,27.940001,21.870102
3,2020-07-07,AAPL,93.852501,94.654999,93.057503,91.233368,112424400.0,1.0,3.019082,92.444331,81.971257,64.558755,121.482696,41.887967,84.483737,78.353269,29.430000,61.119055
4,2020-07-08,AAPL,94.180000,95.375000,94.089996,93.358208,117092000.0,2.0,3.167339,93.375728,81.954913,66.460878,134.303901,44.686147,85.011193,78.797403,28.080000,19.126095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2021-10-21,AAPL,148.809998,149.639999,147.869995,147.575989,61421000.0,3.0,0.370826,147.885239,136.199538,55.568760,102.569171,6.487243,143.056634,145.185103,15.010000,101.005609
331,2021-10-22,AAPL,149.690002,150.179993,148.639999,146.796066,58883400.0,4.0,0.569646,148.209361,136.050162,54.510475,111.334716,9.255682,143.047419,145.238874,15.430000,110.005216
332,2021-10-25,AAPL,148.679993,149.369995,147.619995,146.746704,50720600.0,0.0,0.714987,148.689109,135.893250,54.442589,98.269819,3.092989,143.017473,145.288208,15.240000,22.741301
333,2021-10-26,AAPL,149.330002,150.839996,149.009995,147.418060,60893400.0,1.0,0.874266,149.357354,135.956568,55.227092,126.324899,10.840686,143.056965,145.354317,15.980000,20.607021


# Meta-Learning Model Setup

# FinRL Model Setup

# Integrated Training Pipeline

# Evaluation and Testing

# Results Analysis and Visualization